In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

In [ ]:
hermes_process_state = pd.read_csv('dataset/hermes_process_states.csv')
hermes_process_state.head()

In [ ]:
knowledge_tools = pd.read_csv('dataset/knowledge_tools.csv')
knowledge_tools.head()

In [ ]:
knowledge_analysis_categories = pd.read_csv('dataset/knowledge_analysis_categories.csv')

knowledge_analysis_categories.sort_values(by='escalation_status', ascending=False, inplace=True)
knowledge_analysis_categories

In [ ]:
hermes_process_logs = pd.read_csv('dataset/hermes_process_logs.csv')
hermes_process_logs.head()

In [ ]:
hermes_process_logs.info()

In [ ]:
# check null values
hermes_process_logs.isnull().sum()

In [2]:
# from onsite tool_responce column, there is two type of response
# 1. Message has been sent to staff
# 2. Message not sent because

# Function to determine if message was sent or not
def extract_response_type(response):
    if "Message has been sent to staff" in response:
        return "Sent"
    elif "Message was not sent because" in response:
        return "Not Sent"
    else:
        return "Unknown"

In [3]:
import ast

# Function to extract 'information' from the message
def extract_information(message):
    try:
        message_dict = ast.literal_eval(message)
        return message_dict.get('information', '')
    except (ValueError, SyntaxError):
        return ''

In [19]:
import ast

# Function to extract 'booking id' from the message
def extract_bookingID(data):
    try:
        bookingid_dict = ast.literal_eval(data)
        return bookingid_dict.get('booking_id', '')
    except (ValueError, SyntaxError):
        return 'not available'    

In [5]:
import re

# Define the list of common greetings
# greetings = [
#     "Hi", "Hello", "Dear", "Bonjour", "Halo", 
#     "Good morning", "Good afternoon", "Good evening"
# ]
greetings = [
    "Hi", "Hello", "Dear", "Bonjour", "Halo", 
    "Bonsoir", "Ciao", "Cher", "Liebe", "Goedenavond", 
    "Hej", "Selamat", "Merhaba", "Hai", "Guten Morgen", 
    "Good morning", "Good afternoon", "Good evening"
]

# Create the regex pattern
pattern = re.compile(r"(?:" + "|".join(greetings) + r")\s+([^\n,]+)")

# Function to extract names from messages
def extract_name(message):
    match = pattern.search(message)
    if match:
        return match.group(1).strip()
    return None

# uji coba 1

## tes category that null

In [ ]:
# temp dataset where category is null dan agentname is Hemres
tes = hermes_process_logs[(hermes_process_logs['category_id'].isnull()) & (hermes_process_logs['agent_name'] == 'Hermes')]
tes.drop(columns=['prompt_arguments', 'prompt_response', 'created_at', 'updated_at','state_id','state_name'], inplace=True)

In [ ]:
# remove duplicate rows from tes
tes.duplicated().sum()
tes.drop_duplicates(inplace=True) 


## cek cek column

In [ ]:
# agent name unique values and count
hermes_process_logs['agent_name'].value_counts()

In [ ]:
# groupby category name and count the number of logs horizontally
category_counts = hermes_process_logs.groupby('category_name').size().reset_index(name='counts')
category_counts = category_counts.sort_values('counts', ascending=False)

# Plot the category counts vertically
plt.figure(figsize=(8, 10))  # Adjust figure size as needed
sns.barplot(x='counts', y='category_name', data=category_counts, orient='h')  
plt.xlabel('Counts')
plt.ylabel('Category Name')
plt.title('Number of Logs by Category')
plt.show()



In [ ]:
# copy hermes process logs to a new dataframe
process_logs = hermes_process_logs.copy()

# # drop all null values
# process_logs.dropna(inplace=True)

process_logs['state_name'].value_counts()

#process_logs['category_name'].value_counts().plot(kind='bar')

## check onsite request

In [ ]:
# get data only for category onsite
onsite = hermes_process_logs[hermes_process_logs['category_name'] == 'request_onsite']
onsite

In [ ]:
onsite.info()

In [ ]:
# convert created_at to datetime
onsite['created_at'] = pd.to_datetime(onsite['created_at'])
# convert updated_at to datetime
onsite['updated_at'] = pd.to_datetime(onsite['updated_at'])
onsite.info()

In [ ]:
onsite['escalation_status'].value_counts()

In [ ]:
# show 1 example of data where escalation status is 2
onsite[onsite['escalation_status'] == 2]['category_summary'].iloc[0]

In [ ]:
# check onsite final answer that null
onsite_final_answer_null = onsite[onsite['final_answer'].isnull()]
# check the for specific id 
onsite_final_answer_null.iloc[8]

## check null value

In [ ]:
# Display the columns with null values and their counts
null_counts = onsite.isnull().sum()
print(null_counts[null_counts > 0])

In [ ]:
# check on each row if the final_answer is null and escalation_status is also null
onsite[(onsite['final_answer'].isnull()) & (onsite['escalation_status'].isnull())]

In [ ]:
# count unique id 
onsite['id'].nunique()

In [ ]:
onsite['user_id'].nunique()

In [ ]:
# count the number of logs for each user_id
user_counts = onsite['user_id'].value_counts().reset_index(name='counts')
user_counts = user_counts.sort_values('counts', ascending=False)

# show the top 10 users with the most logs
user_counts.head(10)

## check response send or not send.

In [ ]:
# Ensure the 'tool_response' column is filled with empty strings where there are NaNs
onsite.loc[:,'tool_response'] = onsite['tool_response'].fillna('')

# Apply function to extract response type, and use .loc to avoid SettingWithCopyWarning
onsite = onsite.copy()
onsite.loc[:,'response_type'] = onsite['tool_response'].apply(extract_response_type)

# Count occurrences
sent_count = onsite[onsite['response_type'] == 'Sent'].shape[0]
not_sent_count = onsite[onsite['response_type'] == 'Not Sent'].shape[0]

print("Sent Count:", sent_count)
print("Not Sent Count:", not_sent_count)

In [ ]:
# Plot the response type counts
plt.figure(figsize=(6, 6))
sns.countplot(x='response_type', data=onsite)
plt.xlabel('Response Type')
plt.ylabel('Counts')
plt.title('Onsite Request Response Type')
plt.show()

In [ ]:
# Average response time for sent requests
onsite_sent = onsite[onsite['response_type'] == 'Sent']
onsite_sent.loc[:,'response_time'] = (onsite_sent['updated_at'] - onsite_sent['created_at']).dt.total_seconds()
average_response_time = onsite_sent['response_time'].mean()

# Print counts
print("Sent Count:", sent_count)
print("Not Sent Count:", not_sent_count)
print(f"Average Response Time for Sent Requests: {average_response_time:.2f} seconds")

In [ ]:
# Bar chart for sent vs not sent requests
plt.figure(figsize=(8, 6))
plt.bar(['Sent Requests', 'Not Sent Requests'], [sent_count, not_sent_count], color=['green', 'red'])
plt.title('Sent vs Not Sent Requests')
plt.ylabel('Number of Requests')
plt.show()


In [ ]:
# Recommendations
success_rate = (sent_count / (sent_count + not_sent_count)) * 100
print(f"Success Rate: {success_rate:.2f}%")
if success_rate < 90:
    print("Recommendation: Investigate the causes of unsent requests and improve system reliability.")
else:
    print("System is performing well with a high success rate.")

### check repeat message

In [ ]:
onsite.head()

In [ ]:
onsite[onsite['response_type'] == 'Not Sent'].iloc[50]

In [ ]:
# get 1 example of data where response type is not sent
#onsite[onsite['response_type'] == 'Not Sent'].iloc[0]

# get all log user_id, last_message,tool_response and response type where user_id is HMHWNXDWSD 
check = onsite[onsite['user_id'] == 'HMHWNXDWSD'][['user_id', 'last_message', 'tool_response', 'response_type','created_at','updated_at']].copy()
check.drop_duplicates(inplace=True)
check


In [ ]:
# get 1 example of data where response type is not sent
#onsite[onsite['response_type'] == 'Not Sent'].iloc[0]

# get all log user_id, last_message,tool_response and response type where user_id is HMKR5FD2Y5 
check = onsite[onsite['user_id'] == 'HMKR5FD2Y5'][['user_id', 'last_message','category_summary', 'tool_response', 'response_type','created_at','updated_at']].copy()
check.drop_duplicates(inplace=True)
check


In [ ]:
# get all log user_id, last_message,tool_response and response type where user_id is  HMEREZT9ZH 
check = onsite[onsite['user_id'] == 'HMEREZT9ZH'][['user_id', 'last_message','category_summary', 'tool_response', 'response_type','created_at','updated_at']].copy()
check.drop_duplicates(inplace=True)
check['tool_response'].value_counts()


## check some reqeust 

In [ ]:
# from onsite count and group by state_name
state_counts = onsite['state_name'].value_counts().reset_index(name='counts')
state_counts = state_counts.sort_values('counts', ascending=False)

# Plot the state counts
plt.figure(figsize=(8, 6))
sns.barplot(y='counts', x='state_name', data=state_counts)
plt.ylabel('Counts')
plt.xlabel('State Name')
plt.title('Onsite Request State Counts')

In [ ]:
# find all log for user_id
onsite_user = onsite[onsite['user_id'] ==  'HMHWNXDWSD']

# group by state_name and send/not send
state_response_counts = onsite_user.groupby(['state_name', 'response_type']).size().reset_index(name='counts')
state_response_counts = state_response_counts.sort_values('counts', ascending=False)
state_response_counts

In [ ]:
# get all dataframe where response type is not sent
not_sent = onsite[onsite['response_type'] == 'Not Sent']
not_sent.head()

In [ ]:
# create new column information and apply the function
not_sent.loc[:,'information'] = not_sent['tool_response'].apply(extract_information)

# Display the first few rows of the updated DataFrame
not_sent.head()


In [ ]:
# unique information
not_sent['information'].unique()

## check about unknown response/error

In [ ]:
# there is unknown response type
onsite_unknown = onsite[onsite['response_type'] == 'Unknown']

onsite_unknown.head(20)

## check all tool send to staff

In [ ]:
send_to_staff = hermes_process_logs[hermes_process_logs['tool_id'] == 21.0]
send_to_staff.head()

In [ ]:
send_to_staff['category_name'].value_counts()

In [ ]:
# get request_special_requirements for send to staff
send_to_staff_special_requirements = send_to_staff[send_to_staff['category_name'] == 'request_special_requirements']
send_to_staff_special_requirements

In [ ]:
# filter all message only tool execution
tool_execution = hermes_process_logs[hermes_process_logs['state_name'] == 'tool-execution']

# sort it base on escalation_status
tool_execution = tool_execution.sort_values('escalation_status', ascending=False)

tool_execution

In [ ]:
# use extract information function to get information
tool_execution.loc[:,'information'] = tool_execution['tool_response'].apply(extract_information)

## make a new dataframe

In [ ]:
dataset = onsite.copy()

In [ ]:
dataset.isnull().sum()
dataset.dropna(inplace=True)

In [ ]:
# Step 1: Filter the DataFrame
dataset = dataset[dataset['response_type'] == 'Sent']



In [ ]:
# Step 2: Drop the specified columns
columns_to_drop = [
    'agent_name', 'prompt_arguments', 'prompt_response', 'category_id',
    'category_name', 'category_description', 'tool_id', 'tool_name', 
    'tool_arguments', 'tool_response'
]
dataset.drop(columns=columns_to_drop, inplace=True)

dataset

In [ ]:
# Step 3: drop state_name, state_id, drop duplicate
dataset.drop(columns=['state_name', 'state_id'], inplace=True)
dataset.drop_duplicates(inplace=True)
dataset

In [ ]:
# step 3 make a new column name to save name form final_answer
# Apply the function to the final_answer column and create a new column
dataset['name'] = dataset['final_answer'].apply(extract_name)

dataset

In [ ]:
colum_to_drop2 = ['final_answer']# 'response_type']
dataset.drop(columns=colum_to_drop2, inplace=True)

dataset

In [ ]:
# Step 4: drop if name is null
dataset.dropna(subset=['name'], inplace=True)

dataset.info()



In [ ]:
# sort dataset base on escalation_status
dataset = dataset.sort_values('escalation_status', ascending=False)

In [ ]:
# save as json
dataset.to_json('dataset/onsite_requests.json', orient='records', lines=True)


# uji coba 2

## category null 

In [ ]:
null = hermes_process_logs[hermes_process_logs['category_name'].isnull()]

In [ ]:
duplicated = hermes_process_logs[hermes_process_logs.duplicated()]
duplicated['category_name'].value_counts()

## for early checkin example of repeat message

In [ ]:
early_checkin = duplicated[duplicated['category_name'] == 'request_early_check_in']
early_checkin

In [ ]:
cols = ['last_message', 'category_actions', 'tool_arguments', 'tool_response']

for col in cols:
    print(f"Unique values in {col} column:")
    print(early_checkin[col].unique())
    print()

In [ ]:
# get all row for id HM3J88HXS5 for columns = cols
cols2 = ['last_message','category_name', 'category_summary', 'escalation_status', 'tool_response', 'tool_name', 'created_at']
check1 =hermes_process_logs[hermes_process_logs['user_id'] == 'HM3J88HXS5'][cols2]
check1.duplicated().sum()
check1.drop_duplicates(inplace=True)
check1

In [ ]:
check1.iloc[1]

## 3

In [ ]:
request = hermes_process_logs.copy()
request.dropna(inplace=True)
# filter all row where category_name contains request
request = request[request['category_name'].str.contains('request')]

In [ ]:
request = request[request['state_name'] == 'tool-execution']
request['category_name'].value_counts()

In [ ]:
# extract information from tool_response
request.loc[:,'information'] = request['tool_response'].apply(extract_information)

In [ ]:
cols_to_drop = ['state_id', 'state_name', 'tool_id', 'tool_arguments', 'tool_response','prompt_arguments','prompt_response','category_id','category_description']
request.drop(columns=cols_to_drop, inplace=True)


In [ ]:
request

In [ ]:
# plot the category_name by counts
plt.figure(figsize=(8, 6))
sns.countplot(y='category_name', data=request)
plt.ylabel('Category Name')
plt.xlabel('Counts')
plt.title('Request Categories')
plt.show()

In [ ]:
# save request as json
request.to_csv('dataset/request.csv', index=False)

# uji coba 3

In [6]:
import glob

# Path to the directory containing the CSV files
file_path = 'Bangkit Data/*.csv'

# Load all CSV files
all_files = glob.glob(file_path)

# Initialize an empty list to store DataFrames
dfs = []

# Iterate over the list of files and read each CSV file
for file in all_files:
    df = pd.read_csv(file)
    dfs.append(df)

# Concatenate all DataFrames into a single DataFrame
df = pd.concat(dfs, ignore_index=True)

# Display the shape of the combined DataFrame
df.shape


(103871, 21)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103871 entries, 0 to 103870
Data columns (total 21 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   id                    103871 non-null  int64  
 1   user_id               103871 non-null  object 
 2   agent_name            103871 non-null  object 
 3   last_message          103811 non-null  object 
 4   final_answer          100879 non-null  object 
 5   escalation_status     103419 non-null  float64
 6   prompt_arguments      103205 non-null  object 
 7   prompt_response       103205 non-null  object 
 8   state_id              103205 non-null  float64
 9   state_name            103205 non-null  object 
 10  category_id           84301 non-null   float64
 11  category_name         84301 non-null   object 
 12  category_description  84301 non-null   object 
 13  category_summary      84301 non-null   object 
 14  category_actions      84301 non-null   object 
 15  

In [8]:
df.isnull().sum()

id                          0
user_id                     0
agent_name                  0
last_message               60
final_answer             2992
escalation_status         452
prompt_arguments          666
prompt_response           666
state_id                  666
state_name                666
category_id             19570
category_name           19570
category_description    19570
category_summary        19570
category_actions        19570
tool_id                 20324
tool_name               20324
tool_arguments          20324
tool_response           20450
created_at                  0
updated_at                  0
dtype: int64

In [ ]:
df['category_name'].value_counts()

In [47]:
# Filter rows where the category_name is 'request_onsite'
onsite_requests = df[df['category_name'] == 'request_onsite']

# Display the first few rows of onsite requests
onsite_requests.head()


,id,user_id,agent_name,last_message,final_answer,escalation_status,prompt_arguments,prompt_response,state_id,state_name,...,category_name,category_description,category_summary,category_actions,tool_id,tool_name,tool_arguments,tool_response,created_at,updated_at
25,22722,HMPRWNECHN,Hosting,"Thank you, I left my used towels outside. Can ...","Hi Ianna,\n\nI've noted your request for a bed...",2.0,"{""user"": ""guest-confirmed-HMPRWNECHN-202405100...","{""data"": {""cost"": 46.365, ""completion"": {""inde...",1.0,goal-setting,...,request_onsite,For guest requests related to amenities such a...,The guest requested to have their bedsheets ch...,Coordinate with onsite staff for change of bed...,21.0,send_onsite_request_to_staff,"{""booking_id"": ""HMPRWNECHN"", ""guest_request_in...","{'information': ""Message has been sent to staf...",2024-05-10 00:27:10,2024-05-10 00:27:43
26,22722,HMPRWNECHN,Hosting,"Thank you, I left my used towels outside. Can ...","Hi Ianna,\n\nI've noted your request for a bed...",2.0,"{""user"": ""guest-confirmed-HMPRWNECHN"", ""model""...","{""data"": {""cost"": 43.11, ""completion"": {""index...",2.0,tool-calling,...,request_onsite,For guest requests related to amenities such a...,The guest requested to have their bedsheets ch...,Coordinate with onsite staff for change of bed...,21.0,send_onsite_request_to_staff,"{""booking_id"": ""HMPRWNECHN"", ""guest_request_in...","{'information': ""Message has been sent to staf...",2024-05-10 00:27:10,2024-05-10 00:27:43
27,22722,HMPRWNECHN,Hosting,"Thank you, I left my used towels outside. Can ...","Hi Ianna,\n\nI've noted your request for a bed...",2.0,"{""user"": null, ""model"": ""gpt-3.5-turbo-0125"", ...","{""data"": {""cost"": 1.245, ""completion"": {""index...",5.0,tool-execution,...,request_onsite,For guest requests related to amenities such a...,The guest requested to have their bedsheets ch...,Coordinate with onsite staff for change of bed...,21.0,send_onsite_request_to_staff,"{""booking_id"": ""HMPRWNECHN"", ""guest_request_in...","{'information': ""Message has been sent to staf...",2024-05-10 00:27:10,2024-05-10 00:27:43
28,22722,HMPRWNECHN,Hosting,"Thank you, I left my used towels outside. Can ...","Hi Ianna,\n\nI've noted your request for a bed...",2.0,"{""user"": ""guest-confirmed-HMPRWNECHN"", ""model""...","{""data"": {""cost"": 881.25, ""completion"": {""inde...",3.0,generate-answer,...,request_onsite,For guest requests related to amenities such a...,The guest requested to have their bedsheets ch...,Coordinate with onsite staff for change of bed...,21.0,send_onsite_request_to_staff,"{""booking_id"": ""HMPRWNECHN"", ""guest_request_in...","{'information': ""Message has been sent to staf...",2024-05-10 00:27:10,2024-05-10 00:27:43
29,22722,HMPRWNECHN,Hosting,"Thank you, I left my used towels outside. Can ...","Hi Ianna,\n\nI've noted your request for a bed...",2.0,"{""user"": ""guest-confirmed-HMPRWNECHN"", ""model""...","{""data"": {""cost"": 53.385000000000005, ""complet...",4.0,validate-answer,...,request_onsite,For guest requests related to amenities such a...,The guest requested to have their bedsheets ch...,Coordinate with onsite staff for change of bed...,21.0,send_onsite_request_to_staff,"{""booking_id"": ""HMPRWNECHN"", ""guest_request_in...","{'information': ""Message has been sent to staf...",2024-05-10 00:27:10,2024-05-10 00:27:43


In [ ]:
onsite_requests.info() 

In [ ]:
print(onsite_requests['escalation_status'].value_counts())
print(onsite_requests['state_name'].value_counts())

In [ ]:
# plot the state_name by counts
plt.figure(figsize=(8, 6))
sns.countplot(y='state_name', data=onsite_requests)
plt.ylabel('State Name')
plt.xlabel('Counts')
plt.title('Onsite Request States')
plt.show()

In [48]:
onsite_requests.isnull().sum()
onsite_requests.dropna(inplace=True)

C:\Users\ahini\AppData\Local\Temp\ipykernel_13088\951670900.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  onsite_requests.dropna(inplace=True)


In [33]:
onsite_requests.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9087 entries, 25 to 103870
Data columns (total 21 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    9087 non-null   int64  
 1   user_id               9087 non-null   object 
 2   agent_name            9087 non-null   object 
 3   last_message          9087 non-null   object 
 4   final_answer          9087 non-null   object 
 5   escalation_status     9087 non-null   float64
 6   prompt_arguments      9087 non-null   object 
 7   prompt_response       9087 non-null   object 
 8   state_id              9087 non-null   float64
 9   state_name            9087 non-null   object 
 10  category_id           9087 non-null   float64
 11  category_name         9087 non-null   object 
 12  category_description  9087 non-null   object 
 13  category_summary      9087 non-null   object 
 14  category_actions      9087 non-null   object 
 15  tool_id               9

In [ ]:
# as some message is repeat 5 time for every state name. so filter only 1 state and drop duplicate
onsite_requests = onsite_requests[onsite_requests['state_name'] == 'tool-execution']
onsite_requests.drop_duplicates(inplace=True)
print(onsite_requests.shape)

In [ ]:
# extract response type 
# Ensure the 'tool_response' column is filled with empty strings where there are NaNs
onsite_requests.loc[:,'tool_response'] = onsite_requests['tool_response'].fillna('')

# Apply function to extract response type, and use .loc to avoid SettingWithCopyWarning
onsite_requests = onsite_requests.copy()
onsite_requests.loc[:,'response_type'] = onsite_requests['tool_response'].apply(extract_response_type)

# count the number of sent and not sent
total_sent_requests = onsite_requests[onsite_requests['response_type'] == 'Sent'].shape[0]
total_not_sent_requests = onsite_requests[onsite_requests['response_type'] == 'Not Sent'].shape[0]
total_unkown_requests = onsite_requests[onsite_requests['response_type'] == 'Unknown'].shape[0]

print("Sent Count:", total_sent_requests)
print("Not Sent Count:", total_not_sent_requests)
print("Unknown Count:", total_unkown_requests)

In [ ]:
# Calculate percentages
total_requests = total_sent_requests + total_not_sent_requests
sent_percentage = (total_sent_requests / total_requests) * 100
not_sent_percentage = (total_not_sent_requests / total_requests) * 100

print("Sent Percentage:", sent_percentage)
print("Not Sent Percentage:", not_sent_percentage)


In [ ]:
import matplotlib.pyplot as plt

# Pie chart for sent and not sent requests
labels = ['Sent', 'Not Sent']
sizes = [sent_percentage, not_sent_percentage]
colors = ['#4CAF50', '#FF5733']
explode = (0.1, 0)  # explode the 1st slice

plt.figure(figsize=(6, 6))
plt.pie(sizes, explode=explode, labels=labels, colors=colors,
        autopct='%1.1f%%', shadow=True, startangle=140)

plt.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.
plt.title('Guest Onsite Request Handling')
plt.show()


In [ ]:
onsite_requests.head()

In [ ]:
# get information from tool_response
onsite_requests.loc[:,'information'] = onsite_requests['tool_response'].apply(extract_information)

# get name form final_answer
onsite_requests.loc[:,'name'] = onsite_requests['final_answer'].apply(extract_name)

onsite_requests.head()

In [ ]:
# Convert 'created_at' to datetime
onsite_requests['created_at'] = pd.to_datetime(onsite_requests['created_at'])
onsite_requests.info()

In [ ]:
# # Convert 'created_at' to datetime

# Set the created_at as the index
onsite_requests.set_index('created_at', inplace=True)

# Resample by day and count sent and not sent requests
daily_sent = onsite_requests[onsite_requests['response_type'] == 'Sent'].resample('D').size()
daily_not_sent = onsite_requests[onsite_requests['response_type'] == 'Not Sent'].resample('D').size()

# Plot the frequency over time
plt.figure(figsize=(14, 7))
plt.plot(daily_sent, label='Sent Requests', color='green')
plt.plot(daily_not_sent, label='Not Sent Requests', color='red')
plt.xlabel('Date')
plt.ylabel('Number of Requests')
plt.title('Daily Onsite Requests Sent vs Not Sent')
plt.legend()
plt.show()


In [ ]:
# make new table count of sent and not sent for each created_at
daily_counts = onsite_requests.groupby([onsite_requests.index.date, 'response_type']).size().unstack()
daily_counts.fillna(0, inplace=True)

# sort daily counts by maximum in sent
daily_counts = daily_counts.sort_values('Sent', ascending=False)
daily_counts

In [ ]:
drop_col = ['tool_id', 'tool_name', 'tool_arguments', 'prompt_arguments', 'prompt_response', 'category_id', 'category_description', 'state_name', 'state_id']
onsite_requests.drop(columns=drop_col, inplace=True)
onsite_requests.info()

In [ ]:
# count number of request for each user_id 
user_counts = onsite_requests['user_id'].value_counts().reset_index(name='counts')
user_counts = user_counts.sort_values('counts', ascending=False)
user_counts

In [ ]:
# count of request per user name
user_counts = onsite_requests['name'].value_counts().reset_index(name='counts')
user_counts = user_counts.sort_values('counts', ascending=False)
user_counts

# uji 4 

In [49]:
onsite_requests.head()

,id,user_id,agent_name,last_message,final_answer,escalation_status,prompt_arguments,prompt_response,state_id,state_name,...,category_name,category_description,category_summary,category_actions,tool_id,tool_name,tool_arguments,tool_response,created_at,updated_at
25,22722,HMPRWNECHN,Hosting,"Thank you, I left my used towels outside. Can ...","Hi Ianna,\n\nI've noted your request for a bed...",2.0,"{""user"": ""guest-confirmed-HMPRWNECHN-202405100...","{""data"": {""cost"": 46.365, ""completion"": {""inde...",1.0,goal-setting,...,request_onsite,For guest requests related to amenities such a...,The guest requested to have their bedsheets ch...,Coordinate with onsite staff for change of bed...,21.0,send_onsite_request_to_staff,"{""booking_id"": ""HMPRWNECHN"", ""guest_request_in...","{'information': ""Message has been sent to staf...",2024-05-10 00:27:10,2024-05-10 00:27:43
26,22722,HMPRWNECHN,Hosting,"Thank you, I left my used towels outside. Can ...","Hi Ianna,\n\nI've noted your request for a bed...",2.0,"{""user"": ""guest-confirmed-HMPRWNECHN"", ""model""...","{""data"": {""cost"": 43.11, ""completion"": {""index...",2.0,tool-calling,...,request_onsite,For guest requests related to amenities such a...,The guest requested to have their bedsheets ch...,Coordinate with onsite staff for change of bed...,21.0,send_onsite_request_to_staff,"{""booking_id"": ""HMPRWNECHN"", ""guest_request_in...","{'information': ""Message has been sent to staf...",2024-05-10 00:27:10,2024-05-10 00:27:43
27,22722,HMPRWNECHN,Hosting,"Thank you, I left my used towels outside. Can ...","Hi Ianna,\n\nI've noted your request for a bed...",2.0,"{""user"": null, ""model"": ""gpt-3.5-turbo-0125"", ...","{""data"": {""cost"": 1.245, ""completion"": {""index...",5.0,tool-execution,...,request_onsite,For guest requests related to amenities such a...,The guest requested to have their bedsheets ch...,Coordinate with onsite staff for change of bed...,21.0,send_onsite_request_to_staff,"{""booking_id"": ""HMPRWNECHN"", ""guest_request_in...","{'information': ""Message has been sent to staf...",2024-05-10 00:27:10,2024-05-10 00:27:43
28,22722,HMPRWNECHN,Hosting,"Thank you, I left my used towels outside. Can ...","Hi Ianna,\n\nI've noted your request for a bed...",2.0,"{""user"": ""guest-confirmed-HMPRWNECHN"", ""model""...","{""data"": {""cost"": 881.25, ""completion"": {""inde...",3.0,generate-answer,...,request_onsite,For guest requests related to amenities such a...,The guest requested to have their bedsheets ch...,Coordinate with onsite staff for change of bed...,21.0,send_onsite_request_to_staff,"{""booking_id"": ""HMPRWNECHN"", ""guest_request_in...","{'information': ""Message has been sent to staf...",2024-05-10 00:27:10,2024-05-10 00:27:43
29,22722,HMPRWNECHN,Hosting,"Thank you, I left my used towels outside. Can ...","Hi Ianna,\n\nI've noted your request for a bed...",2.0,"{""user"": ""guest-confirmed-HMPRWNECHN"", ""model""...","{""data"": {""cost"": 53.385000000000005, ""complet...",4.0,validate-answer,...,request_onsite,For guest requests related to amenities such a...,The guest requested to have their bedsheets ch...,Coordinate with onsite staff for change of bed...,21.0,send_onsite_request_to_staff,"{""booking_id"": ""HMPRWNECHN"", ""guest_request_in...","{'information': ""Message has been sent to staf...",2024-05-10 00:27:10,2024-05-10 00:27:43


In [50]:
drop_col = ['tool_id', 'tool_name', 'prompt_arguments', 'prompt_response', 'category_id', 'category_description', 'state_name', 'state_id']
onsite_requests.drop(columns=drop_col, inplace=True)
onsite_requests.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9087 entries, 25 to 103870
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 9087 non-null   int64  
 1   user_id            9087 non-null   object 
 2   agent_name         9087 non-null   object 
 3   last_message       9087 non-null   object 
 4   final_answer       9087 non-null   object 
 5   escalation_status  9087 non-null   float64
 6   category_name      9087 non-null   object 
 7   category_summary   9087 non-null   object 
 8   category_actions   9087 non-null   object 
 9   tool_arguments     9087 non-null   object 
 10  tool_response      9087 non-null   object 
 11  created_at         9087 non-null   object 
 12  updated_at         9087 non-null   object 
dtypes: float64(1), int64(1), object(11)
memory usage: 993.9+ KB


C:\Users\ahini\AppData\Local\Temp\ipykernel_13088\2225462842.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  onsite_requests.drop(columns=drop_col, inplace=True)


In [51]:
onsite_requests.head()

,id,user_id,agent_name,last_message,final_answer,escalation_status,category_name,category_summary,category_actions,tool_arguments,tool_response,created_at,updated_at
25,22722,HMPRWNECHN,Hosting,"Thank you, I left my used towels outside. Can ...","Hi Ianna,\n\nI've noted your request for a bed...",2.0,request_onsite,The guest requested to have their bedsheets ch...,Coordinate with onsite staff for change of bed...,"{""booking_id"": ""HMPRWNECHN"", ""guest_request_in...","{'information': ""Message has been sent to staf...",2024-05-10 00:27:10,2024-05-10 00:27:43
26,22722,HMPRWNECHN,Hosting,"Thank you, I left my used towels outside. Can ...","Hi Ianna,\n\nI've noted your request for a bed...",2.0,request_onsite,The guest requested to have their bedsheets ch...,Coordinate with onsite staff for change of bed...,"{""booking_id"": ""HMPRWNECHN"", ""guest_request_in...","{'information': ""Message has been sent to staf...",2024-05-10 00:27:10,2024-05-10 00:27:43
27,22722,HMPRWNECHN,Hosting,"Thank you, I left my used towels outside. Can ...","Hi Ianna,\n\nI've noted your request for a bed...",2.0,request_onsite,The guest requested to have their bedsheets ch...,Coordinate with onsite staff for change of bed...,"{""booking_id"": ""HMPRWNECHN"", ""guest_request_in...","{'information': ""Message has been sent to staf...",2024-05-10 00:27:10,2024-05-10 00:27:43
28,22722,HMPRWNECHN,Hosting,"Thank you, I left my used towels outside. Can ...","Hi Ianna,\n\nI've noted your request for a bed...",2.0,request_onsite,The guest requested to have their bedsheets ch...,Coordinate with onsite staff for change of bed...,"{""booking_id"": ""HMPRWNECHN"", ""guest_request_in...","{'information': ""Message has been sent to staf...",2024-05-10 00:27:10,2024-05-10 00:27:43
29,22722,HMPRWNECHN,Hosting,"Thank you, I left my used towels outside. Can ...","Hi Ianna,\n\nI've noted your request for a bed...",2.0,request_onsite,The guest requested to have their bedsheets ch...,Coordinate with onsite staff for change of bed...,"{""booking_id"": ""HMPRWNECHN"", ""guest_request_in...","{'information': ""Message has been sent to staf...",2024-05-10 00:27:10,2024-05-10 00:27:43


In [52]:
# extract response type 
onsite_requests.loc[:,'response_type'] = onsite_requests['tool_response'].apply(extract_response_type)

# extract information
onsite_requests.loc[:,'information'] = onsite_requests['tool_response'].apply(extract_information)

# extract name
onsite_requests.loc[:,'name'] = onsite_requests['final_answer'].apply(extract_name)

onsite_requests.head()

C:\Users\ahini\AppData\Local\Temp\ipykernel_13088\3958339398.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  onsite_requests.loc[:,'response_type'] = onsite_requests['tool_response'].apply(extract_response_type)
C:\Users\ahini\AppData\Local\Temp\ipykernel_13088\3958339398.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  onsite_requests.loc[:,'information'] = onsite_requests['tool_response'].apply(extract_information)
C:\Users\ahini\AppData\Local\Temp\ipykernel_13088\3958339398.py:8: SettingWithCopyWa

,id,user_id,agent_name,last_message,final_answer,escalation_status,category_name,category_summary,category_actions,tool_arguments,tool_response,created_at,updated_at,response_type,information,name
25,22722,HMPRWNECHN,Hosting,"Thank you, I left my used towels outside. Can ...","Hi Ianna,\n\nI've noted your request for a bed...",2.0,request_onsite,The guest requested to have their bedsheets ch...,Coordinate with onsite staff for change of bed...,"{""booking_id"": ""HMPRWNECHN"", ""guest_request_in...","{'information': ""Message has been sent to staf...",2024-05-10 00:27:10,2024-05-10 00:27:43,Sent,Message has been sent to staff. Message detail...,Ianna
26,22722,HMPRWNECHN,Hosting,"Thank you, I left my used towels outside. Can ...","Hi Ianna,\n\nI've noted your request for a bed...",2.0,request_onsite,The guest requested to have their bedsheets ch...,Coordinate with onsite staff for change of bed...,"{""booking_id"": ""HMPRWNECHN"", ""guest_request_in...","{'information': ""Message has been sent to staf...",2024-05-10 00:27:10,2024-05-10 00:27:43,Sent,Message has been sent to staff. Message detail...,Ianna
27,22722,HMPRWNECHN,Hosting,"Thank you, I left my used towels outside. Can ...","Hi Ianna,\n\nI've noted your request for a bed...",2.0,request_onsite,The guest requested to have their bedsheets ch...,Coordinate with onsite staff for change of bed...,"{""booking_id"": ""HMPRWNECHN"", ""guest_request_in...","{'information': ""Message has been sent to staf...",2024-05-10 00:27:10,2024-05-10 00:27:43,Sent,Message has been sent to staff. Message detail...,Ianna
28,22722,HMPRWNECHN,Hosting,"Thank you, I left my used towels outside. Can ...","Hi Ianna,\n\nI've noted your request for a bed...",2.0,request_onsite,The guest requested to have their bedsheets ch...,Coordinate with onsite staff for change of bed...,"{""booking_id"": ""HMPRWNECHN"", ""guest_request_in...","{'information': ""Message has been sent to staf...",2024-05-10 00:27:10,2024-05-10 00:27:43,Sent,Message has been sent to staff. Message detail...,Ianna
29,22722,HMPRWNECHN,Hosting,"Thank you, I left my used towels outside. Can ...","Hi Ianna,\n\nI've noted your request for a bed...",2.0,request_onsite,The guest requested to have their bedsheets ch...,Coordinate with onsite staff for change of bed...,"{""booking_id"": ""HMPRWNECHN"", ""guest_request_in...","{'information': ""Message has been sent to staf...",2024-05-10 00:27:10,2024-05-10 00:27:43,Sent,Message has been sent to staff. Message detail...,Ianna


In [53]:
# extract booking id
onsite_requests.loc[:,'booking_id'] = onsite_requests['tool_arguments'].apply(extract_bookingID)

onsite_requests.head()

C:\Users\ahini\AppData\Local\Temp\ipykernel_13088\1608960055.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  onsite_requests.loc[:,'booking_id'] = onsite_requests['tool_arguments'].apply(extract_bookingID)


,id,user_id,agent_name,last_message,final_answer,escalation_status,category_name,category_summary,category_actions,tool_arguments,tool_response,created_at,updated_at,response_type,information,name,booking_id
25,22722,HMPRWNECHN,Hosting,"Thank you, I left my used towels outside. Can ...","Hi Ianna,\n\nI've noted your request for a bed...",2.0,request_onsite,The guest requested to have their bedsheets ch...,Coordinate with onsite staff for change of bed...,"{""booking_id"": ""HMPRWNECHN"", ""guest_request_in...","{'information': ""Message has been sent to staf...",2024-05-10 00:27:10,2024-05-10 00:27:43,Sent,Message has been sent to staff. Message detail...,Ianna,HMPRWNECHN
26,22722,HMPRWNECHN,Hosting,"Thank you, I left my used towels outside. Can ...","Hi Ianna,\n\nI've noted your request for a bed...",2.0,request_onsite,The guest requested to have their bedsheets ch...,Coordinate with onsite staff for change of bed...,"{""booking_id"": ""HMPRWNECHN"", ""guest_request_in...","{'information': ""Message has been sent to staf...",2024-05-10 00:27:10,2024-05-10 00:27:43,Sent,Message has been sent to staff. Message detail...,Ianna,HMPRWNECHN
27,22722,HMPRWNECHN,Hosting,"Thank you, I left my used towels outside. Can ...","Hi Ianna,\n\nI've noted your request for a bed...",2.0,request_onsite,The guest requested to have their bedsheets ch...,Coordinate with onsite staff for change of bed...,"{""booking_id"": ""HMPRWNECHN"", ""guest_request_in...","{'information': ""Message has been sent to staf...",2024-05-10 00:27:10,2024-05-10 00:27:43,Sent,Message has been sent to staff. Message detail...,Ianna,HMPRWNECHN
28,22722,HMPRWNECHN,Hosting,"Thank you, I left my used towels outside. Can ...","Hi Ianna,\n\nI've noted your request for a bed...",2.0,request_onsite,The guest requested to have their bedsheets ch...,Coordinate with onsite staff for change of bed...,"{""booking_id"": ""HMPRWNECHN"", ""guest_request_in...","{'information': ""Message has been sent to staf...",2024-05-10 00:27:10,2024-05-10 00:27:43,Sent,Message has been sent to staff. Message detail...,Ianna,HMPRWNECHN
29,22722,HMPRWNECHN,Hosting,"Thank you, I left my used towels outside. Can ...","Hi Ianna,\n\nI've noted your request for a bed...",2.0,request_onsite,The guest requested to have their bedsheets ch...,Coordinate with onsite staff for change of bed...,"{""booking_id"": ""HMPRWNECHN"", ""guest_request_in...","{'information': ""Message has been sent to staf...",2024-05-10 00:27:10,2024-05-10 00:27:43,Sent,Message has been sent to staff. Message detail...,Ianna,HMPRWNECHN


In [54]:
# check if booking id is same as user id
onsite_requests['booking_id_match'] = onsite_requests['booking_id'] == onsite_requests['user_id']
onsite_requests['booking_id_match'].value_counts()

C:\Users\ahini\AppData\Local\Temp\ipykernel_13088\1112803014.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  onsite_requests['booking_id_match'] = onsite_requests['booking_id'] == onsite_requests['user_id']


booking_id_match
True     8945
False     142
Name: count, dtype: int64

In [55]:
# check for booking id match is false is response type is unknown
onsite_requests[onsite_requests['booking_id_match'] == False]['response_type'].value_counts()

response_type
Unknown    142
Name: count, dtype: int64

In [56]:
# drop where booking id match is false
onsite_requests = onsite_requests[onsite_requests['booking_id_match'] == True]
onsite_requests.drop(columns=['booking_id_match'], inplace=True)
onsite_requests.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8945 entries, 25 to 103870
Data columns (total 17 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 8945 non-null   int64  
 1   user_id            8945 non-null   object 
 2   agent_name         8945 non-null   object 
 3   last_message       8945 non-null   object 
 4   final_answer       8945 non-null   object 
 5   escalation_status  8945 non-null   float64
 6   category_name      8945 non-null   object 
 7   category_summary   8945 non-null   object 
 8   category_actions   8945 non-null   object 
 9   tool_arguments     8945 non-null   object 
 10  tool_response      8945 non-null   object 
 11  created_at         8945 non-null   object 
 12  updated_at         8945 non-null   object 
 13  response_type      8945 non-null   object 
 14  information        8945 non-null   object 
 15  name               8281 non-null   object 
 16  booking_id         8945 no

In [57]:
new_df = onsite_requests.copy()
new_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8945 entries, 25 to 103870
Data columns (total 17 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 8945 non-null   int64  
 1   user_id            8945 non-null   object 
 2   agent_name         8945 non-null   object 
 3   last_message       8945 non-null   object 
 4   final_answer       8945 non-null   object 
 5   escalation_status  8945 non-null   float64
 6   category_name      8945 non-null   object 
 7   category_summary   8945 non-null   object 
 8   category_actions   8945 non-null   object 
 9   tool_arguments     8945 non-null   object 
 10  tool_response      8945 non-null   object 
 11  created_at         8945 non-null   object 
 12  updated_at         8945 non-null   object 
 13  response_type      8945 non-null   object 
 14  information        8945 non-null   object 
 15  name               8281 non-null   object 
 16  booking_id         8945 no

In [58]:
new_df.dropna(inplace=True)
new_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8281 entries, 25 to 103870
Data columns (total 17 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 8281 non-null   int64  
 1   user_id            8281 non-null   object 
 2   agent_name         8281 non-null   object 
 3   last_message       8281 non-null   object 
 4   final_answer       8281 non-null   object 
 5   escalation_status  8281 non-null   float64
 6   category_name      8281 non-null   object 
 7   category_summary   8281 non-null   object 
 8   category_actions   8281 non-null   object 
 9   tool_arguments     8281 non-null   object 
 10  tool_response      8281 non-null   object 
 11  created_at         8281 non-null   object 
 12  updated_at         8281 non-null   object 
 13  response_type      8281 non-null   object 
 14  information        8281 non-null   object 
 15  name               8281 non-null   object 
 16  booking_id         8281 no

In [59]:
new_df['response_type'].value_counts()

response_type
Sent        5662
Not Sent    2488
Unknown      131
Name: count, dtype: int64

In [60]:
unknown = new_df[new_df['response_type'] == 'Unknown']
unknown

,id,user_id,agent_name,last_message,final_answer,escalation_status,category_name,category_summary,category_actions,tool_arguments,tool_response,created_at,updated_at,response_type,information,name,booking_id
2952,23149,1788695501,Hosting,Let me know the contact person who will pick m...,"Hello Ravindra,\n\nThank you for reaching out,...",2.0,request_onsite,Ravindra is asking for the contact person who ...,Provide contact details for pick-up person and...,"{""booking_id"": ""1788695501"", ""guest_request_in...","{'information': {'error': True, 'data': {'book...",2024-05-10 11:34:12,2024-05-10 11:34:38,Unknown,"{'error': True, 'data': {'booking_id': '178869...",Ravindra,1788695501
2953,23149,1788695501,Hosting,Let me know the contact person who will pick m...,"Hello Ravindra,\n\nThank you for reaching out,...",2.0,request_onsite,Ravindra is asking for the contact person who ...,Provide contact details for pick-up person and...,"{""booking_id"": ""1788695501"", ""guest_request_in...","{'information': {'error': True, 'data': {'book...",2024-05-10 11:34:12,2024-05-10 11:34:38,Unknown,"{'error': True, 'data': {'booking_id': '178869...",Ravindra,1788695501
2954,23149,1788695501,Hosting,Let me know the contact person who will pick m...,"Hello Ravindra,\n\nThank you for reaching out,...",2.0,request_onsite,Ravindra is asking for the contact person who ...,Provide contact details for pick-up person and...,"{""booking_id"": ""1788695501"", ""guest_request_in...","{'information': {'error': True, 'data': {'book...",2024-05-10 11:34:12,2024-05-10 11:34:38,Unknown,"{'error': True, 'data': {'booking_id': '178869...",Ravindra,1788695501
2955,23149,1788695501,Hosting,Let me know the contact person who will pick m...,"Hello Ravindra,\n\nThank you for reaching out,...",2.0,request_onsite,Ravindra is asking for the contact person who ...,Provide contact details for pick-up person and...,"{""booking_id"": ""1788695501"", ""guest_request_in...","{'information': {'error': True, 'data': {'book...",2024-05-10 11:34:12,2024-05-10 11:34:38,Unknown,"{'error': True, 'data': {'booking_id': '178869...",Ravindra,1788695501
2956,23149,1788695501,Hosting,Let me know the contact person who will pick m...,"Hello Ravindra,\n\nThank you for reaching out,...",2.0,request_onsite,Ravindra is asking for the contact person who ...,Provide contact details for pick-up person and...,"{""booking_id"": ""1788695501"", ""guest_request_in...","{'information': {'error': True, 'data': {'book...",2024-05-10 11:34:12,2024-05-10 11:34:38,Unknown,"{'error': True, 'data': {'booking_id': '178869...",Ravindra,1788695501
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103866,22717,1788695501,Hosting,We do provide a pickup service from the harbor...,"Hello Ravindra,\n\nWe have acknowledged your p...",2.0,request_onsite,"User, Ravindra Shimpi, requested a pickup serv...",Provide information and confirm the pick-up re...,"{""booking_id"": ""1788695501"", ""guest_request_in...","{'information': {'error': True, 'data': {'book...",2024-05-09 23:57:11,2024-05-09 23:57:53,Unknown,"{'error': True, 'data': {'booking_id': '178869...",Ravindra,1788695501
103867,22717,1788695501,Hosting,We do provide a pickup service from the harbor...,"Hello Ravindra,\n\nWe have acknowledged your p...",2.0,request_onsite,"User, Ravindra Shimpi, requested a pickup serv...",Provide information and confirm the pick-up re...,"{""booking_id"": ""1788695501"", ""guest_request_in...","{'information': {'error': True, 'data': {'book...",2024-05-09 23:57:11,2024-05-09 23:57:53,Unknown,"{'error': True, 'data': {'booking_id': '178869...",Ravindra,1788695501
103868,22717,1788695501,Hosting,We do provide a pickup service from the harbor...,"Hello Ravindra,\n\nWe have acknowledged your p...",2.0,request_onsite,"User, Ravindra Shimpi, requested a pickup serv...",Provide information and confirm the pick-up re...,"{""booking_id"": ""1788695501"", ""guest_request_in...","{'information': {'error': True, 'data':

In [61]:
# new df drop unknown
new_df = new_df[new_df['response_type'] != 'Unknown']
new_df['response_type'].value_counts()

response_type
Sent        5662
Not Sent    2488
Name: count, dtype: int64

In [63]:
drop_col = ['tool_response', 'tool_arguments','final_answer','category_name']
new_df.drop(columns=drop_col, inplace=True)

In [64]:
new_df.duplicated().sum()

6968

In [65]:
new_df.drop_duplicates(inplace=True)

In [66]:
new_df['response_type'].value_counts()

response_type
Sent        857
Not Sent    325
Name: count, dtype: int64

In [68]:
# get unique information where response type is not sent
new_df[new_df['response_type'] == 'Not Sent']['information'].unique()

array(['Message was not sent because it has been sent to on-site staff before.'],
      dtype=object)

In [71]:
new_df

,id,user_id,agent_name,last_message,escalation_status,category_summary,category_actions,created_at,updated_at,response_type,information,name,booking_id
25,22722,HMPRWNECHN,Hosting,"Thank you, I left my used towels outside. Can ...",2.0,The guest requested to have their bedsheets ch...,Coordinate with onsite staff for change of bed...,2024-05-10 00:27:10,2024-05-10 00:27:43,Sent,Message has been sent to staff. Message detail...,Ianna,HMPRWNECHN
203,22742,HMKBE38MJE,Hosting,I clicked.on the link now not working,2.0,The user is asking about the category.,Ask for more details or context about the imag...,2024-05-10 01:26:57,2024-05-10 01:27:34,Sent,Message has been sent to staff. Message detail...,Khaled,HMKBE38MJE
399,22768,HMXJSDJ3F4,Hosting,"Hi!\nI forgot I phone charger, do you have one?",2.0,The guest is inquiring about needing an iPhone...,Offer assistance for the guest's forgotten iPh...,2024-05-10 02:20:52,2024-05-10 02:21:13,Sent,Message has been sent to staff. Message detail...,Léonard,HMXJSDJ3F4
421,22773,HMEX9CCPRT,Hosting,Do you have option to provide scooter and/or b...,2.0,Alain reported that lights in two rooms are no...,Assist with the lights issue in two rooms and ...,2024-05-10 02:24:56,2024-05-10 02:25:50,Sent,Message has been sent to staff. Message detail...,Alain,HMEX9CCPRT
422,22773,HMEX9CCPRT,Hosting,Do you have option to provide scooter and/or b...,2.0,Alain reported that lights in two rooms are no...,Assist with the lights issue in two rooms and ...,2024-05-10 02:24:56,2024-05-10 02:25:50,Not Sent,Message was not sent because it has been sent ...,Alain,HMEX9CCPRT
...,...,...,...,...,...,...,...,...,...,...,...,...,...
103671,22697,HMRJAAJSBY,Hosting,Yes I am sitting here waiting,2.0,"A Rigters is unable to lock the door, expressi...",Assist in resolving the door locking issue and...,2024-05-09 18:54:42,2024-05-09 18:55:31,Not Sent,Message was not sent because it has been sent ...,A,HMRJAAJSBY
103745,22705,HMRJAAJSBY,Hosting,Is someone comingng,2.0,The guest is expressing concern as nobody has ...,Assist the guest with immediate on-site suppor...,2024-05-09 20:34:11,2024-05-09 20:34:49,Not Sent,Message was not sent because it has been sent ...,A,HMRJAAJSBY
103764,22707,HMRJAAJSBY,Hosting,I hope so because I’m still sitting here waiting,2.0,A Rigters is experiencing an ongoing door issu...,Provide prompt on-site assistance and resoluti...,2024-05-09 20:49:15,2024-05-09 20:50:15,Not Sent,Message was not sent because it has been sent ...,A,HMRJAAJSBY
103765,22707,HMRJAAJSBY,Hosting,I hope so because I’m still sitting here waiting,2.0,A Rigters is experiencing an ongoing door issu...,Provide prompt on-site assistance and resoluti...,2024-05-09 20:49:15,2024-05-09 20:50:15,Sent,Message has been sent to staff. Message detail...,A,HMRJAAJSBY


In [72]:
# save new_df as json
new_df.to_json('dataset/onsite_requests.json', orient='records', lines=True)